# E-Commerce Analytics Demo with Moltres

This notebook demonstrates a complete end-to-end analytics workflow using Moltres, from database setup to advanced analytics queries.

## Scenario: E-Commerce Sales Analytics

We'll build a realistic e-commerce database with:

- **Customers**: Customer profiles and demographics
- **Products**: Product catalog with categories and pricing
- **Orders**: Order transactions with timestamps
- **Order Items**: Line items for each order

Then we'll perform real-world analytics:

- Revenue analysis by product and category
- Customer lifetime value (CLV) calculations
- Monthly sales trends
- Top customers and products
- Conversion and retention metrics


In [1]:
# Import required librariesfrom moltres import connect, colfrom moltres.table.schema import columnfrom moltres.io.records import Recordsfrom moltres.expressions import functions as Fimport randomfrom datetime import datetime, timedelta# For visualization (optional)try:    import pandas as pd    import matplotlib.pyplot as plt    HAS_VIZ = Trueexcept ImportError:    HAS_VIZ = False    print("⚠️  pandas/matplotlib not installed. Install for visualization: pip install pandas matplotlib seaborn")print("✅ Imports successful!")

✅ Imports successful!


## 1. Database Setup

We'll use DuckDB for this demo - it's perfect for analytics workloads and supports SQL operations efficiently.


In [2]:
# Connect to DuckDB (in-memory database, perfect for analytics)
db = connect("duckdb:///:memory:")

print("✅ Connected to DuckDB")
print(f"Database type: {type(db).__name__}")

✅ Connected to DuckDB
Database type: Database


## 2. Create Database Schema

Let's create our e-commerce tables with proper schema definitions.


In [3]:
# Create Customers table
db.create_table(
    "customers",
    [
        column("customer_id", "INTEGER", primary_key=True),
        column("first_name", "VARCHAR(50)", nullable=False),
        column("last_name", "VARCHAR(50)", nullable=False),
        column("email", "VARCHAR(100)", nullable=False),
        column("city", "VARCHAR(50)"),
        column("state", "VARCHAR(2)"),
        column("country", "VARCHAR(50)", default="USA"),
        column("registration_date", "DATE", nullable=False),
        column("customer_segment", "VARCHAR(20)"),  # e.g., "Premium", "Standard", "Basic"
    ],
).collect()

# Create Products table
db.create_table(
    "products",
    [
        column("product_id", "INTEGER", primary_key=True),
        column("product_name", "VARCHAR(100)", nullable=False),
        column("category", "VARCHAR(50)", nullable=False),
        column("subcategory", "VARCHAR(50)"),
        column("price", "DECIMAL(10, 2)", nullable=False),
        column("cost", "DECIMAL(10, 2)"),  # Cost for margin calculations
        column("in_stock", "BOOLEAN", default=True),
    ],
).collect()

# Create Orders table
db.create_table(
    "orders",
    [
        column("order_id", "INTEGER", primary_key=True),
        column("customer_id", "INTEGER", nullable=False),
        column("order_date", "TIMESTAMP", nullable=False),
        column("status", "VARCHAR(20)", default="completed"),  # completed, cancelled, pending
        column("shipping_cost", "DECIMAL(10, 2)", default=0.0),
        column("tax_amount", "DECIMAL(10, 2)", default=0.0),
    ],
).collect()

# Create Order Items table (line items for each order)
db.create_table(
    "order_items",
    [
        column("order_item_id", "INTEGER", primary_key=True),
        column("order_id", "INTEGER", nullable=False),
        column("product_id", "INTEGER", nullable=False),
        column("quantity", "INTEGER", nullable=False),
        column("unit_price", "DECIMAL(10, 2)", nullable=False),
        column("discount", "DECIMAL(10, 2)", default=0.0),
    ],
).collect()

print("✅ All tables created successfully!")

✅ All tables created successfully!


In [4]:
# Set random seed for reproducibility
random.seed(42)

# Generate customer data
first_names = [
    "Alice",
    "Bob",
    "Charlie",
    "Diana",
    "Eve",
    "Frank",
    "Grace",
    "Henry",
    "Ivy",
    "Jack",
    "Kate",
    "Liam",
    "Mia",
    "Noah",
    "Olivia",
    "Paul",
    "Quinn",
    "Rachel",
    "Sam",
    "Tina",
    "Uma",
    "Victor",
    "Wendy",
    "Xavier",
    "Yara",
    "Zoe",
]
last_names = [
    "Smith",
    "Johnson",
    "Williams",
    "Brown",
    "Jones",
    "Garcia",
    "Miller",
    "Davis",
    "Rodriguez",
    "Martinez",
    "Hernandez",
    "Lopez",
    "Wilson",
    "Anderson",
    "Thomas",
    "Taylor",
    "Moore",
    "Jackson",
    "Martin",
    "Lee",
]
cities = [
    "New York",
    "Los Angeles",
    "Chicago",
    "Houston",
    "Phoenix",
    "Philadelphia",
    "San Antonio",
    "San Diego",
    "Dallas",
    "San Jose",
]
states = ["NY", "CA", "IL", "TX", "AZ", "PA", "TX", "CA", "TX", "CA"]
segments = ["Premium", "Standard", "Basic"]
categories = ["Electronics", "Clothing", "Home & Garden", "Sports", "Books", "Toys"]
subcategories = {
    "Electronics": ["Laptops", "Phones", "Tablets", "Accessories"],
    "Clothing": ["Men's", "Women's", "Kids", "Accessories"],
    "Home & Garden": ["Furniture", "Decor", "Tools", "Outdoor"],
    "Sports": ["Fitness", "Outdoor", "Team Sports", "Water Sports"],
    "Books": ["Fiction", "Non-Fiction", "Educational", "Children's"],
    "Toys": ["Action Figures", "Board Games", "Puzzles", "Educational"],
}

# Generate 100 customers
customers_data = []
base_date = datetime(2020, 1, 1)
for i in range(1, 101):
    reg_date = base_date + timedelta(days=random.randint(0, 1460))  # Random date in last 4 years
    customers_data.append(
        {
            "customer_id": i,
            "first_name": random.choice(first_names),
            "last_name": random.choice(last_names),
            "email": f"customer{i}@example.com",
            "city": random.choice(cities),
            "state": random.choice(states),
            "country": "USA",
            "registration_date": reg_date.strftime("%Y-%m-%d"),
            "customer_segment": random.choice(segments),
        }
    )

# Insert customers
Records.from_list(customers_data, database=db).insert_into("customers")
print(f"✅ Inserted {len(customers_data)} customers")

✅ Inserted 100 customers


In [5]:
# Generate product catalog
products_data = []
product_id = 1
for category in categories:
    for subcat in subcategories[category]:
        # Create 3-5 products per subcategory
        num_products = random.randint(3, 5)
        for _ in range(num_products):
            # Realistic pricing based on category
            if category == "Electronics":
                base_price = random.uniform(200, 2000)
            elif category == "Clothing":
                base_price = random.uniform(20, 200)
            elif category == "Home & Garden":
                base_price = random.uniform(50, 500)
            elif category == "Sports":
                base_price = random.uniform(30, 300)
            elif category == "Books":
                base_price = random.uniform(10, 50)
            else:  # Toys
                base_price = random.uniform(15, 150)

            cost = base_price * random.uniform(0.4, 0.7)  # Cost is 40-70% of price

            products_data.append(
                {
                    "product_id": product_id,
                    "product_name": f"{subcat} Product {product_id}",
                    "category": category,
                    "subcategory": subcat,
                    "price": round(base_price, 2),
                    "cost": round(cost, 2),
                    "in_stock": random.choice([True, True, True, False]),  # 75% in stock
                }
            )
            product_id += 1

# Insert products
Records.from_list(products_data, database=db).insert_into("products")
print(f"✅ Inserted {len(products_data)} products")
print(f"   Categories: {', '.join(categories)}")

✅ Inserted 99 products
   Categories: Electronics, Clothing, Home & Garden, Sports, Books, Toys


In [6]:
# Generate orders and order items
# Create orders over the past 2 years with realistic patterns
orders_data = []
order_items_data = []
order_id = 1
order_item_id = 1

start_date = datetime(2022, 1, 1)
end_date = datetime(2024, 1, 1)
current_date = start_date

# Generate ~500 orders
while current_date < end_date and order_id <= 500:
    # Some days have more orders (weekends, holidays)
    orders_per_day = random.choices([0, 1, 2, 3, 4, 5], weights=[10, 30, 30, 20, 8, 2])[0]

    for _ in range(orders_per_day):
        customer_id = random.randint(1, 100)
        order_date = current_date + timedelta(
            hours=random.randint(8, 22), minutes=random.randint(0, 59)
        )

        # Order status (mostly completed, some cancelled)
        status = random.choices(["completed", "cancelled", "pending"], weights=[85, 10, 5])[0]
        shipping_cost = round(random.uniform(5, 25), 2)
        tax_rate = random.uniform(0.06, 0.10)  # 6-10% tax

        orders_data.append(
            {
                "order_id": order_id,
                "customer_id": customer_id,
                "order_date": order_date.strftime("%Y-%m-%d %H:%M:%S"),
                "status": status,
                "shipping_cost": shipping_cost,
                "tax_amount": 0,  # Will calculate after items
            }
        )

        # Each order has 1-5 items
        num_items = random.randint(1, 5)
        order_subtotal = 0

        for _ in range(num_items):
            product = random.choice(products_data)
            quantity = random.randint(1, 3)
            discount = (
                round(random.uniform(0, 0.2), 2) if random.random() < 0.3 else 0
            )  # 30% chance of discount
            unit_price = float(product["price"]) * (1 - float(discount))

            order_items_data.append(
                {
                    "order_item_id": order_item_id,
                    "order_id": order_id,
                    "product_id": product["product_id"],
                    "quantity": quantity,
                    "unit_price": round(unit_price, 2),
                    "discount": round(discount * product["price"], 2),
                }
            )

            order_subtotal += unit_price * quantity
            order_item_id += 1

        # Update tax amount based on subtotal
        orders_data[-1]["tax_amount"] = round(order_subtotal * tax_rate, 2)

        order_id += 1

    # Move to next day
    current_date += timedelta(days=1)

# Insert orders
Records.from_list(orders_data, database=db).insert_into("orders")
print(f"✅ Inserted {len(orders_data)} orders")

# Insert order items
Records.from_list(order_items_data, database=db).insert_into("order_items")
print(f"✅ Inserted {len(order_items_data)} order items")

print("\n📊 Data Summary:")
print(f"   Customers: {len(customers_data)}")
print(f"   Products: {len(products_data)}")
print(f"   Orders: {len(orders_data)}")
print(f"   Order Items: {len(order_items_data)}")

✅ Inserted 501 orders
✅ Inserted 1528 order items

📊 Data Summary:
   Customers: 100
   Products: 99
   Orders: 501
   Order Items: 1528


## 4. Basic Data Exploration

Let's start with some basic queries to understand our data.


In [7]:
# View sample customers
customers_df = db.table("customers").select().limit(10)
print("Sample Customers:")
for row in customers_df.collect():
    print(
        f"  {row['first_name']} {row['last_name']} ({row['email']}) - {row['city']}, {row['state']} - {row['customer_segment']}"
    )

Sample Customers:
  Diana Smith (customer1@example.com) - Phoenix, TX - Premium
  Xavier Brown (customer2@example.com) - Dallas, CA - Basic
  Bob Smith (customer3@example.com) - Los Angeles, TX - Premium
  Tina Smith (customer4@example.com) - Dallas, TX - Basic
  Wendy Jackson (customer5@example.com) - San Antonio, TX - Standard
  Ivy Smith (customer6@example.com) - Chicago, TX - Standard
  Eve Miller (customer7@example.com) - Philadelphia, CA - Premium
  Diana Lopez (customer8@example.com) - Philadelphia, CA - Standard
  Xavier Thomas (customer9@example.com) - Dallas, CA - Standard
  Rachel Martinez (customer10@example.com) - San Jose, PA - Basic


In [ ]:
# View sample products
products_df = db.table("products").select().where(col("category") == "Electronics").limit(5)
print("Sample Electronics Products:")
for row in products_df.collect():
    price = float(row["price"]) if row["price"] is not None else 0
    cost = float(row["cost"]) if row["cost"] is not None else 0
    print(f"  {row['product_name']} - ${price:.2f} (Cost: ${cost:.2f})")

In [ ]:
# Check order statistics
total_orders = db.table("orders").select().count()
completed_orders = db.table("orders").select().where(col("status") == "completed").count()
print(f"Total Orders: {total_orders}")
print(f"Completed Orders: {completed_orders}")
print(f"Completion Rate: {completed_orders / total_orders * 100:.1f}%")

## 5. Revenue Analysis

Let's calculate total revenue, revenue by category, and identify top-selling products.


In [ ]:
# Calculate total revenue from completed orders
order_items_df = db.table("order_items").select()
orders_df = db.table("orders").select()
revenue_df = order_items_df.join(
    orders_df.where(col("status") == "completed"), on="order_id", how="inner"
).select(
    (col("quantity") * col("unit_price").cast("DECIMAL", precision=10, scale=2)).alias(
        "item_revenue"
    )
)
total_revenue = revenue_df.select(F.sum(col("item_revenue")).alias("total_revenue")).collect()[0][
    "total_revenue"
]
print(f"💰 Total Revenue: ${total_revenue:,.2f}")

In [ ]:
# Revenue by product category
order_items_df = db.table("order_items").select()
orders_df = db.table("orders").select()
products_df = db.table("products").select()
revenue_by_category = (
    order_items_df.join(orders_df.where(col("status") == "completed"), on="order_id", how="inner")
    .join(products_df, on="product_id", how="inner")
    .select(
        col("category"),
        (col("quantity") * col("unit_price").cast("DECIMAL", precision=10, scale=2)).alias(
            "item_revenue"
        ),
    )
    .group_by("category")
    .agg(F.sum(col("item_revenue")).alias("total_revenue"))
    .order_by(col("total_revenue").desc())
)
print("📊 Revenue by Category:")
for row in revenue_by_category.collect():
    print(f"  {row['category']}: ${row['total_revenue']:,.2f}")

In [ ]:
# Top 10 products by revenue
order_items_df = db.table("order_items").select()
orders_df = db.table("orders").select()
products_df = db.table("products").select()
top_products = (
    order_items_df.join(orders_df.where(col("status") == "completed"), on="order_id", how="inner")
    .join(products_df, on="product_id", how="inner")
    .select(col("product_name"), col("category"), col("quantity"), col("unit_price"))
    .group_by("product_name", "category")
    .agg(
        F.sum(col("quantity") * col("unit_price").cast("DECIMAL", precision=10, scale=2)).alias(
            "total_revenue"
        ),
        F.sum(col("quantity")).alias("units_sold"),
    )
    .order_by(col("total_revenue").desc())
    .limit(10)
)
print("🏆 Top 10 Products by Revenue:")
for i, row in enumerate(top_products.collect(), 1):
    print(f"  {i}. {row['product_name']} ({row['category']})")
    print(f"     Revenue: ${row['total_revenue']:,.2f} | Units Sold: {row['units_sold']}")

## 6. Customer Analytics

Analyze customer behavior, lifetime value, and segmentation.


In [ ]:
# Customer Lifetime Value (CLV) - total revenue per customer
customers_df = db.table("customers").select()
orders_df = db.table("orders").select()
order_items_df = db.table("order_items").select()
customer_clv = (
    customers_df.join(orders_df.where(col("status") == "completed"), on="customer_id", how="inner")
    .join(order_items_df, on="order_id", how="inner")
    .select(
        col("customer_id"),
        col("first_name"),
        col("last_name"),
        col("customer_segment"),
        (col("quantity") * col("unit_price").cast("DECIMAL", precision=10, scale=2)).alias(
            "item_revenue"
        ),
    )
    .group_by("customer_id", "first_name", "last_name", "customer_segment")
    .agg(F.sum(col("item_revenue")).alias("lifetime_value"), F.count("*").alias("order_count"))
    .order_by(col("lifetime_value").desc())
    .limit(10)
)
print("👥 Top 10 Customers by Lifetime Value:")
for i, row in enumerate(customer_clv.collect(), 1):
    print(f"  {i}. {row['first_name']} {row['last_name']} ({row['customer_segment']})")
    print(f"     CLV: ${row['lifetime_value']:,.2f} | Orders: {row['order_count']}")

In [ ]:
# Average order value by customer segment
customers_df = db.table("customers").select()
orders_df = db.table("orders").select()
order_items_df = db.table("order_items").select()
avg_order_by_segment = (
    customers_df.join(orders_df.where(col("status") == "completed"), on="customer_id", how="inner")
    .join(order_items_df, on="order_id", how="inner")
    .select(
        col("customer_segment"),
        col("order_id"),
        (col("quantity") * col("unit_price").cast("DECIMAL", precision=10, scale=2)).alias(
            "item_revenue"
        ),
    )
    .group_by("customer_segment", "order_id")
    .agg(F.sum(col("item_revenue")).alias("order_value"))
    .group_by("customer_segment")
    .agg(F.avg(col("order_value")).alias("avg_order_value"))
    .order_by(col("avg_order_value").desc())
)
print("📈 Average Order Value by Customer Segment:")
for row in avg_order_by_segment.collect():
    print(f"  {row['customer_segment']}: ${row['avg_order_value']:,.2f}")

## 7. Time Series Analysis

Analyze sales trends over time - monthly revenue, order volume, and growth rates.


In [ ]:
# Monthly revenue trend
orders_df = db.table("orders").select()
order_items_df = db.table("order_items").select()
monthly_revenue = (
    orders_df.where(col("status") == "completed")
    .join(order_items_df, on="order_id", how="inner")
    .select(
        F.date_trunc("month", col("order_date")).alias("month"),
        (col("quantity") * col("unit_price").cast("DECIMAL", precision=10, scale=2)).alias(
            "item_revenue"
        ),
    )
    .group_by("month")
    .agg(F.sum(col("item_revenue")).alias("monthly_revenue"), F.count("*").alias("order_count"))
    .order_by(col("month"))
)
print("📅 Monthly Revenue Trend:")
results = monthly_revenue.collect()
for row in results:
    month_str = row["month"][:7] if isinstance(row["month"], str) else str(row["month"])[:7]
    print(f"  {month_str}: ${row['monthly_revenue']:,.2f} ({row['order_count']} orders)")
if HAS_VIZ:
    # Convert to pandas for visualization
    df = pd.DataFrame(results)
    if "month" in df.columns:
        df["month"] = pd.to_datetime(df["month"])
        df = df.sort_values("month")

        plt.figure(figsize=(12, 5))
        plt.plot(df["month"], df["monthly_revenue"], marker="o", linewidth=2, markersize=6)
        plt.title("Monthly Revenue Trend", fontsize=14, fontweight="bold")
        plt.xlabel("Month", fontsize=12)
        plt.ylabel("Revenue ($)", fontsize=12)
        plt.grid(True, alpha=0.3)
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()

In [ ]:
# Revenue by day of week (to identify peak shopping days)
orders_df = db.table("orders").select()
order_items_df = db.table("order_items").select()
daily_revenue = (
    orders_df.where(col("status") == "completed")
    .join(order_items_df, on="order_id", how="inner")
    .select(
        F.dayofweek(col("order_date")).alias("day_of_week"),
        (col("quantity") * col("unit_price").cast("DECIMAL", precision=10, scale=2)).alias(
            "item_revenue"
        ),
    )
    .group_by("day_of_week")
    .agg(F.sum(col("item_revenue")).alias("total_revenue"), F.count("*").alias("order_count"))
    .order_by(col("day_of_week"))
)
day_names = ["Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"]
print("📆 Revenue by Day of Week:")
for row in daily_revenue.collect():
    day_idx = int(row["day_of_week"]) - 1  # Adjust for 1-based indexing
    day_name = day_names[day_idx] if 0 <= day_idx < 7 else f"Day {row['day_of_week']}"
    print(f"  {day_name}: ${row['total_revenue']:,.2f} ({row['order_count']} orders)")

## 8. Profitability Analysis

Calculate profit margins and identify most profitable products.


In [ ]:
# Profit by product (revenue - cost)
order_items_df = db.table("order_items").select()
orders_df = db.table("orders").select()
products_df = db.table("products").select()
product_profitability = (
    order_items_df.join(orders_df, on="order_id", how="inner")
    .join(products_df, on="product_id", how="inner")
    .where(col("status") == "completed")
    .select(
        col("product_name"),
        col("category"),
        col("price"),
        col("cost"),
        col("quantity"),
        col("unit_price"),
    )
    .group_by("product_name", "category")
    .agg(
        F.sum(col("quantity") * col("unit_price").cast("DECIMAL", precision=10, scale=2)).alias(
            "total_revenue"
        ),
        F.sum(col("quantity") * col("cost").cast("DECIMAL", precision=10, scale=2)).alias(
            "total_cost"
        ),
    )
    .select(
        col("product_name"),
        col("category"),
        col("total_revenue"),
        col("total_cost"),
        (col("total_revenue") - col("total_cost")).alias("profit"),
        ((col("total_revenue") - col("total_cost")) / col("total_revenue") * 100).alias(
            "margin_pct"
        ),
    )
    .order_by(col("profit").desc())
    .limit(10)
)
print("💵 Top 10 Most Profitable Products:")
for i, row in enumerate(product_profitability.collect(), 1):
    margin = float(row.get("margin_pct", 0)) if row.get("margin_pct") is not None else 0
    print(f"  {i}. {row['product_name']} ({row['category']})")
    print(f"     Profit: ${row['profit']:,.2f} | Margin: {margin:.1f}%")

## 9. Geographic Analysis

Analyze sales by location to identify top markets.


In [ ]:
# Revenue by state
customers_df = db.table("customers").select()
orders_df = db.table("orders").select()
order_items_df = db.table("order_items").select()
revenue_by_state = (
    customers_df.join(orders_df, on="customer_id", how="inner")
    .join(order_items_df, on="order_id", how="inner")
    .where(col("status") == "completed")
    .select(
        col("state"),
        (col("quantity") * col("unit_price").cast("DECIMAL", precision=10, scale=2)).alias(
            "item_revenue"
        ),
    )
    .group_by("state")
    .agg(F.sum(col("item_revenue")).alias("total_revenue"), F.count("*").alias("order_count"))
    .order_by(col("total_revenue").desc())
    .limit(10)
)
print("🗺️  Top 10 States by Revenue:")
for i, row in enumerate(revenue_by_state.collect(), 1):
    print(f"  {i}. {row['state']}: ${row['total_revenue']:,.2f} ({row['order_count']} orders)")

## 10. Advanced Analytics: Basket Analysis

Identify which products are frequently purchased together.


In [ ]:
# Find products that appear together in the same order# This is a simplified basket analysis - in production you'd use more sophisticated algorithms# We need to do a self-join on order_items to find products in the same order# Get all order items with product info for completed ordersfrom collections import defaultdictorder_items_with_products = (    db.table("order_items")    .select()    .join(        db.table("products").select(),        on="product_id",        how="inner"    )    .join(        db.table("orders").select().where(col("status") == "completed"),        on="order_id",        how="inner"    )    .select(        col("order_id"),        col("product_id"),        col("product_name"),        col("category")    ))# Create two copies with different aliases using subqueries# We'll use a simpler approach: collect and process in Python for this demo# (In production, you'd use a more sophisticated SQL approach)# Get all order-item pairsall_items = order_items_with_products.collect()# Group by order_id and find pairsorder_products = defaultdict(list)for item in all_items:    order_products[item['order_id']].append({        'product_id': item['product_id'],        'product_name': item['product_name'],        'category': item['category']    })# Find co-occurrencesco_occurrences = defaultdict(int)for order_id, products in order_products.items():    # For each pair of products in the same order    for i, p1 in enumerate(products):        for p2 in products[i+1:]:            # Use sorted pair to avoid duplicates (A-B same as B-A)            pair = tuple(sorted([p1['product_name'], p2['product_name']]))            co_occurrences[pair] += 1# Get top 10 pairstop_pairs = sorted(co_occurrences.items(), key=lambda x: x[1], reverse=True)[:10]print("🛒 Top Product Pairs (Frequently Bought Together):")for i, ((p1, p2), count) in enumerate(top_pairs, 1):    print(f"  {i}. {p1} + {p2}")    print(f"     Appeared together: {count} times")

## 11. Data Export and Visualization

Export results to different formats and create visualizations.


In [ ]:
# Export monthly revenue to CSV using DataFrame write API
orders_df = db.table("orders").select()
order_items_df = db.table("order_items").select()
monthly_revenue_df = (
    orders_df.where(col("status") == "completed")
    .join(order_items_df, on="order_id", how="inner")
    .select(
        F.date_trunc("month", col("order_date")).alias("month"),
        (col("quantity") * col("unit_price").cast("DECIMAL", precision=10, scale=2)).alias(
            "item_revenue"
        ),
    )
    .group_by("month")
    .agg(F.sum(col("item_revenue")).alias("monthly_revenue"), F.count("*").alias("order_count"))
    .order_by(col("month"))
)
# Export to CSV
monthly_revenue_df.write.mode("overwrite").format("csv").option("header", True).save(
    "notebooks/monthly_revenue.csv"
)
print("✅ Exported monthly revenue to notebooks/monthly_revenue.csv")

In [ ]:
# If pandas is available, create a comprehensive summary DataFrame
if HAS_VIZ:
    # Re-run queries for visualization
    order_items_df = db.table("order_items").select()
    orders_df = db.table("orders").select()
    products_df = db.table("products").select()
    customers_df = db.table("customers").select()

    # Revenue by category
    revenue_by_category = (
        order_items_df.join(
            orders_df.where(col("status") == "completed"), on="order_id", how="inner"
        )
        .join(products_df, on="product_id", how="inner")
        .select(
            col("category"),
            (col("quantity") * col("unit_price").cast("DECIMAL", precision=10, scale=2)).alias(
                "item_revenue"
            ),
        )
        .group_by("category")
        .agg(F.sum(col("item_revenue")).alias("total_revenue"))
    )

    # Average order value by segment
    avg_order_by_segment = (
        customers_df.join(
            orders_df.where(col("status") == "completed"), on="customer_id", how="inner"
        )
        .join(order_items_df, on="order_id", how="inner")
        .select(
            col("customer_segment"),
            col("order_id"),
            (col("quantity") * col("unit_price").cast("DECIMAL", precision=10, scale=2)).alias(
                "item_revenue"
            ),
        )
        .group_by("customer_segment", "order_id")
        .agg(F.sum(col("item_revenue")).alias("order_value"))
        .group_by("customer_segment")
        .agg(F.avg(col("order_value")).alias("avg_order_value"))
    )

    # Monthly revenue
    monthly_revenue = (
        orders_df.where(col("status") == "completed")
        .join(order_items_df, on="order_id", how="inner")
        .select(
            F.date_trunc("month", col("order_date")).alias("month"),
            (col("quantity") * col("unit_price").cast("DECIMAL", precision=10, scale=2)).alias(
                "item_revenue"
            ),
        )
        .group_by("month")
        .agg(F.sum(col("item_revenue")).alias("monthly_revenue"))
    )

    # Revenue by state
    revenue_by_state = (
        customers_df.join(
            orders_df.where(col("status") == "completed"), on="customer_id", how="inner"
        )
        .join(order_items_df, on="order_id", how="inner")
        .select(
            col("state"),
            (col("quantity") * col("unit_price").cast("DECIMAL", precision=10, scale=2)).alias(
                "item_revenue"
            ),
        )
        .group_by("state")
        .agg(F.sum(col("item_revenue")).alias("total_revenue"))
    )

    # Create a summary dashboard
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))

    # 1. Revenue by Category (Bar Chart)
    cat_revenue = revenue_by_category.collect()
    cat_df = pd.DataFrame(cat_revenue)
    axes[0, 0].bar(cat_df["category"], cat_df["total_revenue"], color="steelblue")
    axes[0, 0].set_title("Revenue by Category", fontweight="bold")
    axes[0, 0].set_ylabel("Revenue ($)")
    axes[0, 0].tick_params(axis="x", rotation=45)

    # 2. Average Order Value by Segment (Bar Chart)
    seg_avg = avg_order_by_segment.collect()
    seg_df = pd.DataFrame(seg_avg)
    axes[0, 1].bar(seg_df["customer_segment"], seg_df["avg_order_value"], color="coral")
    axes[0, 1].set_title("Average Order Value by Segment", fontweight="bold")
    axes[0, 1].set_ylabel("Average Order Value ($)")

    # 3. Monthly Revenue Trend (Line Chart)
    monthly = monthly_revenue.collect()
    monthly_df = pd.DataFrame(monthly)
    if "month" in monthly_df.columns:
        monthly_df["month"] = pd.to_datetime(monthly_df["month"])
        monthly_df = monthly_df.sort_values("month")
        axes[1, 0].plot(
            monthly_df["month"],
            monthly_df["monthly_revenue"],
            marker="o",
            linewidth=2,
            markersize=6,
            color="green",
        )
        axes[1, 0].set_title("Monthly Revenue Trend", fontweight="bold")
        axes[1, 0].set_ylabel("Revenue ($)")
        axes[1, 0].tick_params(axis="x", rotation=45)
        axes[1, 0].grid(True, alpha=0.3)

    # 4. Top States (Horizontal Bar Chart)
    states_rev = revenue_by_state.collect()
    states_df = pd.DataFrame(states_rev)
    axes[1, 1].barh(states_df["state"], states_df["total_revenue"], color="purple")
    axes[1, 1].set_title("Top States by Revenue", fontweight="bold")
    axes[1, 1].set_xlabel("Revenue ($)")

    plt.tight_layout()
    plt.savefig("notebooks/ecommerce_dashboard.png", dpi=150, bbox_inches="tight")
    print("✅ Saved dashboard visualization to notebooks/ecommerce_dashboard.png")
    plt.show()
else:
    print(
        "⚠️  Install pandas and matplotlib for visualization: pip install pandas matplotlib seaborn"
    )

## 12. Summary and Key Insights

Let's create a final summary of key business metrics.


In [ ]:
# Calculate key business metrics
total_customers = db.table("customers").select().count()
active_customers = (
    db.table("customers")
    .select()
    .join(db.table("orders").select(), on="customer_id", how="inner")
    .where(col("status") == "completed")
    .select(col("customer_id"))
    .distinct()
    .count()
)
orders_df = db.table("orders").select()
order_items_df = db.table("order_items").select()
avg_order_value = (
    orders_df.join(order_items_df, on="order_id", how="inner")
    .where(col("status") == "completed")
    .select(
        col("order_id"),
        (col("quantity") * col("unit_price").cast("DECIMAL", precision=10, scale=2)).alias(
            "item_revenue"
        ),
    )
    .group_by("order_id")
    .agg(F.sum(col("item_revenue")).alias("order_value"))
    .select(F.avg(col("order_value")).alias("avg_order_value"))
    .collect()[0]["avg_order_value"]
)
total_products = db.table("products").select().count()
print("=" * 70)
print("📊 E-COMMERCE ANALYTICS SUMMARY")
print("=" * 70)
print("\n👥 Customers:")
print(f"   Total Customers: {total_customers}")
print(f"   Active Customers (with orders): {active_customers}")
print(f"   Customer Activation Rate: {active_customers / total_customers * 100:.1f}%")
print("\n💰 Financial Metrics:")
print(f"   Total Revenue: ${total_revenue:,.2f}")
print(f"   Average Order Value: ${avg_order_value:,.2f}")
print(f"   Total Orders: {completed_orders}")
print("\n📦 Products:")
print(f"   Total Products: {total_products}")
print(f"   Categories: {len(categories)}")
print("\n🎯 Top Performing Category:")
top_cat = revenue_by_category.collect()[0]
print(f"   {top_cat['category']}: ${top_cat['total_revenue']:,.2f}")
print("\n" + "=" * 70)
print("✅ Analysis Complete!")
print("=" * 70)

## Next Steps

This demo showcased:

1. ✅ **Database Setup**: Creating tables with proper schemas
2. ✅ **Data Generation**: Realistic sample data generation
3. ✅ **Basic Queries**: Filtering, selecting, and aggregating data
4. ✅ **Joins**: Combining data from multiple tables
5. ✅ **Revenue Analysis**: Calculating totals, by category, and top products
6. ✅ **Customer Analytics**: CLV, segmentation, and behavior analysis
7. ✅ **Time Series**: Monthly trends and day-of-week patterns
8. ✅ **Profitability**: Margin calculations and profit analysis
9. ✅ **Geographic Analysis**: Sales by location
10. ✅ **Advanced Analytics**: Basket analysis for product recommendations
11. ✅ **Data Export**: Exporting results to CSV
12. ✅ **Visualization**: Creating charts and dashboards

### Real-World Applications:

- **Business Intelligence**: Generate reports for stakeholders
- **Marketing**: Identify high-value customers for campaigns
- **Inventory**: Optimize stock levels based on sales patterns
- **Pricing**: Adjust prices based on profitability analysis
- **Recommendations**: Use basket analysis for cross-selling

### Try These Extensions:

- Add more complex window functions for running totals
- Implement customer cohort analysis
- Calculate churn rates and retention metrics
- Build predictive models using the exported data
- Integrate with real-time data streams
